# <a>Classes Desbalanceadas: você está fazendo errado!</a>



---

O Let's Data produz conteúdos gratuitos sobre Data Science!

Confira algumas das nossa produções:

* [Livro Digital - Como se tornar um Cientista de Dados?](https://ebook.letsdata.ai/)
* [Inscreva-se no canal no YouTube](https://www.youtube.com/c/LetsDataAI?sub_confirmation=1)
* [Siga-nos no Instagram](https://www.instagram.com/letsdata.ai)
* Ouça nosso Podcast no [Spotify](https://open.spotify.com/show/0VsNN95jsJVRS424eCFDlg?si=EvQ1KBq1SqGVD4ojHrx4eQ), [Google Podcast](https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkcy5idXp6c3Byb3V0LmNvbS8xNzMxMjUyLnJzcw), [Apple Podcasts](https://podcasts.apple.com/us/podcast/lets-data-podcast/id1559072194), [Castbox](https://castbox.fm/vic/1559072194), [Deezer](https://www.deezer.com/show/2770422) e demais players. 
* Também estamos no [LinkedIn](https://linkedin.com/company/lets-data), [Twitter](https://twitter.com/letsdataAI), [Facebook](https://www.facebook.com/letsdataAI/).
* [Pra finalizar, nosso blog no Medium](https://medium.com/lets-data).

---

In [ ]:
# Está no requirements mas também está aqui
!pip notebook pandas scikit-learn imblearn

In [1]:
import pandas as pd

In [2]:
# Indios da etnia PIMA
df = pd.read_csv('./data/diabetes.csv')

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [4]:
268/len(df) * 100

34.89583333333333

In [5]:
100 - (268/len(df) * 100)

65.10416666666667

In [6]:
X = df.drop('Outcome', axis=1)
y = df[['Outcome']]

X.shape, y.shape

((768, 8), (768, 1))

In [7]:
from sklearn.model_selection import train_test_split

# Separação de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y)

X_treino.shape, X_teste.shape, y_treino.shape, y_teste.shape

((576, 8), (192, 8), (576, 1), (192, 1))

In [14]:
# SMOTE está disponível no pacote imblearn (lá tem um monte de outras técnicas)
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# função de validação cruzada NA MÃO
def validacao_cruzada(modelo, X, y, oversampling=False):
    # KFold do scikit-learn
    kfold = KFold(n_splits=10)

    # lista de acuracias de cada split
    acuracias_split = []
    
    # iterando sobre os splits
    for idx, (idx_treino, idx_validacao) in enumerate(kfold.split(X)):
        X_split_treino = X.iloc[idx_treino, :]
        y_split_treino = y.iloc[idx_treino, :]
    
        # oversampling, só no split de treino!!
        if oversampling:
            sm = SMOTE(random_state=42)
            X_split_treino, y_split_treino = sm.fit_resample(X_split_treino, y_split_treino)
        
        # Com os dados balenceados SÓ NO TREINO, vamos treinar o nosso modelo
        modelo.fit(X_split_treino, y_split_treino.values.flatten())
    
        X_split_validacao = X.iloc[idx_validacao, :]
        y_split_validacao = y.iloc[idx_validacao, :]
        
        # Validação SEM oversampling, como a cartilha ensina :)
        # Amostra do mundo real, ou seja, com dados DESBALANCEADOS
        predicoes_validacao = modelo.predict(X_split_validacao)
        
        acuracia_split = accuracy_score(y_split_validacao, predicoes_validacao)
        
        acuracias_split.append(acuracia_split)
        
        print(f'Acurácia do split {idx}: {acuracia_split}')
    
    return acuracias_split
    

In [9]:
from sklearn.ensemble import HistGradientBoostingClassifier
from statistics import mean

# Criando o modelo
modelo_hgb = HistGradientBoostingClassifier()

In [15]:
media_acuracia_sem_smote = mean(validacao_cruzada(modelo_hgb, X_treino, y_treino, oversampling=False)) * 100

Acurácia do split 0: 0.8103448275862069
Acurácia do split 1: 0.7586206896551724
Acurácia do split 2: 0.7413793103448276
Acurácia do split 3: 0.7758620689655172
Acurácia do split 4: 0.7068965517241379
Acurácia do split 5: 0.7931034482758621
Acurácia do split 6: 0.7543859649122807
Acurácia do split 7: 0.7894736842105263
Acurácia do split 8: 0.7192982456140351
Acurácia do split 9: 0.631578947368421


In [16]:
media_acuracia_sem_smote

74.80943738656988

In [17]:
media_acuracia_com_smote = mean(validacao_cruzada(modelo_hgb, X_treino, y_treino, oversampling=True)) * 100

Acurácia do split 0: 0.7413793103448276
Acurácia do split 1: 0.7241379310344828
Acurácia do split 2: 0.7413793103448276
Acurácia do split 3: 0.7931034482758621
Acurácia do split 4: 0.6896551724137931
Acurácia do split 5: 0.7931034482758621
Acurácia do split 6: 0.7894736842105263
Acurácia do split 7: 0.7894736842105263
Acurácia do split 8: 0.7719298245614035
Acurácia do split 9: 0.631578947368421


In [18]:
media_acuracia_com_smote

74.65214761040532

In [19]:
f'Sem smote: {media_acuracia_sem_smote:.02f}, com_smote: {media_acuracia_com_smote:.02f}'

'Sem smote: 74.81, com_smote: 74.65'

## CUIDADO! Agora vamos mostrar o jeito ERRADO!

In [20]:
from sklearn.model_selection import cross_val_score

# ERRADO! Não se deve balancear as classes antes do split
sm = SMOTE(random_state=42)
X_treino, y_treino = sm.fit_resample(X_treino, y_treino)

# Os splits criados no cross_val_score estão com a validação BALANCEADA
# E isso é distorcer a realidade!
mean(cross_val_score(modelo_hgb, X_treino, y_treino.values.flatten())) * 100

80.03131991051454